In [42]:
import os
os.environ['USER_AGENT'] = 'MyApp/1.0 (Custom LangChain Application)'
import numpy as np
import pandas as pd

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader, CSVLoader, UnstructuredCSVLoader

In [2]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
query_result = embeddings.embed_query('저는 배가 고파요.')
print(query_result)

[-0.015310164541006088, -0.0272151380777359, 0.01789705455303192, -0.024945009499788284, -0.036770790815353394, 0.006638804450631142, -0.03611087054014206, -0.010875496082007885, -0.01981082558631897, -0.015613728202879429, -0.014993403106927872, 0.01764628477394581, -0.008249010890722275, -0.025340963155031204, 0.010129786096513271, -0.007265728898346424, 0.00982622243463993, -0.007028157357126474, 0.008704356849193573, -0.01747470535337925, 0.00022416688443627208, -0.015191379003226757, 0.018992522731423378, -0.011304444633424282, -0.0006846679607406259, 0.00864496361464262, 0.013620767742395401, -0.015666522085666656, -0.0062263538129627705, -0.004150902386754751, 0.03597888723015785, -0.016471626237034798, 0.0013940829085186124, 0.005635724402964115, 0.01360756903886795, -0.00386383687146008, -0.0005832050810568035, 0.0036526622716337442, -0.005668720696121454, -0.0030438851099461317, -0.02739991620182991, -0.010637925006449223, 0.009846019558608532, -0.020708316937088966, 0.012954

In [3]:
def get_embedding(text):
    return embeddings.embed_query(text)

In [4]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])
df['embedding'] = df.apply(lambda row: get_embedding(row.text), axis=1)
df

,text,embedding
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ..."
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,..."
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009272306226193905, -0.01461657416075468, ..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.017441052943468094, -0.0031565295066684484..."
4,저는 빠른 비트를 좋아해요,"[-0.037991348654031754, -0.013411780819296837,..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.017488501965999603, -0.02157001383602619, ..."


In [5]:
def cos_sim(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(query)
    df['similarity'] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    top_3_doc = df.sort_values(by='similarity', ascending=False).head(3)

    return top_3_doc

In [6]:
sim_result = return_answer_candidate(df, '과일 값이 비싸다.')
sim_result

,text,embedding,similarity
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009272306226193905, -0.01461657416075468, ...",0.824511
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,...",0.814506
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ...",0.805363


In [7]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'})

In [8]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])
df['embedding'] = df.apply(lambda row: get_embedding(row.text), axis=1)
df

,text,embedding
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,..."
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0..."
2,전통 시장에는 다양한 물품들을 팔아요,"[0.0001013833170873113, 0.0005807742127217352,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.0077268523164093494, 0.021975157782435417,..."
4,저는 빠른 비트를 좋아해요,"[-0.019412308931350708, 0.012223082594573498, ..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,..."


In [9]:
sim_result = return_answer_candidate(df, '과일 값이 비싸다.')
sim_result

,text,embedding,similarity
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0...",0.647129
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,...",0.614578
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,...",0.578653


In [10]:
loader = WebBaseLoader('https://docs.langchain.com/langsmith/')
loader_multiple_pages = WebBaseLoader(
    ['https://docs.langchain.com/oss/python/langchain/overview', 'https://docs.langchain.com/oss/python/langgraph/overview']
)

In [11]:
single_doc = loader.load()
print(single_doc[0].metadata)

{'source': 'https://docs.langchain.com/langsmith/', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}


In [12]:
docs = loader_multiple_pages.load()
print(docs[0].page_content)

LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewLangChain v1.0Release notesMigration guideGet startedInstallQuickstartPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryUse in productionStudioTestDeployAgent Chat UIObservabilityOn this page Install Create an agent Core benefitsLangChain overviewCopy pageCopy pageLangChain v1.0 is now available!For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide.If you encounter any issues or have feedback, please open an issue so we can improve. To vi

In [16]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
pages = loader.load_and_split()
print(len(pages))

83


In [18]:
pages[10]
print(pages[10].page_content)

5 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
표Ⅰ-1. 공급 확대 정책 주요 내용 
날짜 주요 내용 
2023년 9월 
공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련 
사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액 
기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진 
원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원 
비아파트 자금 조달 지원 및 규제 개선 
도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선 
2024년 1월 
도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의 
신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화 
다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선 
및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원 
신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고 
건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장 
공공 인수, 재정 조기 집행 및 민자 사업 확대 
 
자료: 국토교통부 보도자료 요약 
 
■ 2024년 주택시장 3대 변수는 공급과 금리, 그리고 정책 
주택시장이 상승과 하락을 반복하면서 주택 경기 불확실성이 확대되고 있다. 특히 매수 수요 위축으
로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 시각이 팽배하다. 무엇보다 여전히 
높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리로 인한 대출 부담은 주택 수요를 크
게 위축시킬 수밖에 없기 때문이다. 
 
그림Ⅰ-11. 매수우위지수 추이  표Ⅰ-2. 주요 금융기관 주택담보대출 금리(2024년 2월) 
 
 
상품명 
당월 
최

In [19]:
loader = PyMuPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
pages = loader.load_and_split()
print(len(pages))

83


In [21]:
pages[10]
print(pages[10].page_content)
print(pages[10].metadata)

5 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
표Ⅰ-1. 공급 확대 정책 주요 내용 
날짜 
주요 내용 
2023년 9월 
공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련 
사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액 
기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진 
원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원 
비아파트 자금 조달 지원 및 규제 개선 
도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선 
2024년 1월 
도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의 
신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화 
다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선 
및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원 
신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고 
건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장 
공공 인수, 재정 조기 집행 및 민자 사업 확대 
자료: 국토교통부 보도자료 요약 
 
■ 2024년 주택시장 3대 변수는 공급과 금리, 그리고 정책 
주택시장이 상승과 하락을 반복하면서 주택 경기 불확실성이 확대되고 있다. 특히 매수 수요 위축으
로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 시각이 팽배하다. 무엇보다 여전히 
높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리로 인한 대출 부담은 주택 수요를 크
게 위축시킬 수밖에 없기 때문이다. 
 
그림Ⅰ-11. 매수우위지수 추이 
표Ⅰ-2. 주요 금융기관 주택담보대출 금리(2024년 2월) 
 
 
상품명 
당월 
최저 


In [23]:
loader = PDFPlumberLoader('./data/2024_KB_부동산_보고서_최종.pdf')
pages = loader.load_and_split()
print(len(pages))

83


In [24]:
pages[10]
print(pages[10].page_content)
print(pages[10].metadata)

2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망
표Ⅰ-1. 공급 확대 정책 주요 내용
날짜 주요 내용
공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련
사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액
기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진
2023년 9월
원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원
비아파트 자금 조달 지원 및 규제 개선
도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선
도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의
신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화
다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선
2024년 1월 및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원
신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고
건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장
공공 인수, 재정 조기 집행 및 민자 사업 확대
자료: 국토교통부 보도자료 요약
■ 2024년 주택시장 3대 변수는 공급과 금리, 그리고 정책
주택시장이 상승과 하락을 반복하면서 주택 경기 불확실성이 확대되고 있다. 특히 매수 수요 위축으
로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 시각이 팽배하다. 무엇보다 여전히
높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리로 인한 대출 부담은 주택 수요를 크
게 위축시킬 수밖에 없기 때문이다.
그림Ⅰ-11. 매수우위지수 추이 표Ⅰ-2. 주요 금융기관 주택담보대출 금리(2024년 2월)
당월 당월 전월
상품명
최저 최고 평균
KB주택담보대출 4.24 5.14 4.85
신한주

In [27]:
loader = CSVLoader('./data/서울시_부동산_실거래가_정보.csv', encoding='utf-8')
documents = loader.load()
print(len(documents))

2001


In [29]:
print(documents[5].metadata)
print(documents[5].page_content)

{'source': './data/서울시_부동산_실거래가_정보.csv', 'row': 5}
﻿접수연도: 2024
자치구코드: 11410
자치구명: 서대문구
법정동코드: 11600
법정동명: 창천동
지번구분: 
지번구분명: 
본번: 
부번: 
건물명: 
계약일: 20241031
물건금액(만원): 340000
건물면적(㎡): 421.83
토지면적(㎡): 284
층: 
권리구분: 
취소일: 
건축년도: 2014
건물용도: 단독다가구
신고구분: 직거래
신고한 개업공인중개사 시군구명: 


In [30]:
loader = UnstructuredCSVLoader('./data/서울시_부동산_실거래가_정보.csv', encoding='utf-8', mode='elements')
documents = loader.load()
print(len(documents))

1


In [34]:
print(str(documents[0].metadata)[:500])
print(str(documents[0].page_content)[:500])

{'source': './data/서울시_부동산_실거래가_정보.csv', 'file_directory': './data', 'filename': '서울시_부동산_실거래가_정보.csv', 'last_modified': '2025-05-18T15:23:45', 'text_as_html': "<table><tr><td>접수연도</td><td>자치구코드</td><td>자치구명</td><td>법정동코드</td><td>법정동명</td><td>지번구분</td><td>지번구분명</td><td>본번</td><td>부번</td><td>건물명</td><td>계약일</td><td>물건금액(만원)</td><td>건물면적(㎡)</td><td>토지면적(㎡)</td><td>층</td><td>권리구분</td><td>취소일</td><td>건축년도</td><td>건물용도</td><td>신고구분</td><td>신고한 개업공인중개사 시군구명</td></tr><tr><td>2024</td><td>11440</td><td>
접수연도 자치구코드 자치구명 법정동코드 법정동명 지번구분 지번구분명 본번 부번 건물명 계약일 물건금액(만원) 건물면적(㎡) 토지면적(㎡) 층 권리구분 취소일 건축년도 건물용도 신고구분 신고한 개업공인중개사 시군구명 2024 11440 마포구 11100 신수동 1 대지 228 3 마인빌 20241031 37300 32.48 20 4 2022 연립다세대 직거래 2024 11320 도봉구 10800 도봉동 1 대지 565 9 (565-9) 20241031 9900 25.92 21 4 2003 연립다세대 직거래 2024 11500 강서구 10900 방화동 1 대지 620 214 (620-214) 20241031 10000 27.37 40.37 7 2011 오피스텔 중개거래 서울 강서구 2024 11680 강남구 11200 자곡동 1 대지 658 0 강남힐스테이트에코 20241031 17000 24.118 31.983 10 2014 오피스텔 중개거래 서울 강남구 2024 11500 강서구 

In [36]:
loader = PyMuPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
pages = loader.load()
print(f'총 글자 수: {len(''.join([i.page_content for i in pages]))}')

총 글자 수: 82396


In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)
print(f'분할된 청크의 수: {len(texts)}')

분할된 청크의 수: 219


In [41]:
print(texts[0])
print(texts[0].page_content)

page_content='1 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
 
Executive Summary 1 
2024년 주택시장 하향 안정 전망, 금리와 공급, 그리고 정책 3대 변수 주목 
주택경기가 상승과 하락을 반복하면서 주택 경기의 불확실성이 확대되고 있으나. 완만한 하향 조정 
가능성이 큰 상황이다. 매수 수요 위축으로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 
시각이 팽배하다. 무엇보다 여전히 높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리 
부담은 주택 수요를 크게 위축시킬 수밖에 없기 때문이다. 다만 주택시장의 주요 변수들의 상황에 따라 
소폭 반등 혹은 하락폭 확대 등의 방향이 정해질 것으로 보인다. 
2024년 주택시장의 주요 변수는 공급과 금리다. 급격하게 위축된 주택 공급이 단기간에 증가하기는 
쉽지 않으나 정부의 공급 시그널이 지속된다면 일정 부분 해소가 될 가능성이 있다. 무엇보다 금리가 주요' metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'source': './data/2024_KB_부동산_보고서_최종.pdf', 'file_path': './data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'format': 'PDF 1.5', 'title': 'Morning Meeting', 'author': '손은경', 'subject': '', 'keywords': '', 'moddate': '2024-03-04T15:30:01+09:00', 'trapped': '', 'modDate': "D:20240304153001+09'00'", 'creationDate': "D:20240304153001+09'00'", 'page': 1}
1 
2024 KB 부동산 보고서:

In [43]:
text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings())
chunks = text_splitter.split_documents(pages)
print(f'분할된 청크의 수: {len(texts)}')

분할된 청크의 수: 219


In [44]:
print(chunks[1].page_content)

1 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
 
Executive Summary 1 
2024년 주택시장 하향 안정 전망, 금리와 공급, 그리고 정책 3대 변수 주목 
주택경기가 상승과 하락을 반복하면서 주택 경기의 불확실성이 확대되고 있으나. 완만한 하향 조정 
가능성이 큰 상황이다.


In [45]:
print(chunks[2].page_content)

매수 수요 위축으로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 
시각이 팽배하다. 무엇보다 여전히 높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리 
부담은 주택 수요를 크게 위축시킬 수밖에 없기 때문이다. 다만 주택시장의 주요 변수들의 상황에 따라 
소폭 반등 혹은 하락폭 확대 등의 방향이 정해질 것으로 보인다. 2024년 주택시장의 주요 변수는 공급과 금리다. 급격하게 위축된 주택 공급이 단기간에 증가하기는 
쉽지 않으나 정부의 공급 시그널이 지속된다면 일정 부분 해소가 될 가능성이 있다. 무엇보다 금리가 주요 
변수가 될 것이다. 기준금리 인하 시기와 인하 폭에 따라 주택 수요는 영향을 받을 수밖에 없기 때문이다. 한편, 수요 위축으로 거래가 급감한 상황에서 실수요자 금융 지원, 관련 규제 완화 등 수요 회복을 위한 
정부 정책도 중요한 변수가 될 전망이다. 7대 이슈를 통해 바라보는 2024년 주택시장 
1 
역대 최저 수준이 지속되고 있는 주택 거래 
 
주택 매매 거래는 2022년에 이어 2년 연속 침체. 총선 이후 정책 불확실성 해소와 금리 인하로 인한 회
복 가능성이 있으나 일부 지역 수요 쏠림 현상과 금리 인하 속도가 더딜 경우 회복세는 제한적일 전망 
2 
주택공급 급격한 감소로 인한 공급 부족 가능성 
 
분양물량과 함께 장기적인 주택 공급 기반인 인허가물량까지 급감. 청약 수요 위축으로 분양 지연이 장기
화될 가능성이 높은 가운데 정부의 공급 정책 구체화가 매우 중요 
3 
노후계획도시 특별법과 재건축 시장 영향 
 
2023년 말 국회를 통과한 「노후계획도시 특별법」 시행을 앞두고 당초 51곳이었던 대상 지역이 108곳으
로 확대. 단기적 효과는 제한적이나 사업진행이 구체화되면 시장 영향도 커질 것  
4 
전세 수요 아파트 집중, 입주물량 부족으로 가격 상승 가능성 확대 
 
비아파트 전세 사기와 보증금 미반환 이슈 등으로 아파트로 전세 수요가 집중되는 가운데 수도권을 중심
으로 아파트 입주물량이 크게

In [48]:
text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings(), breakpoint_threshold_type='standard_deviation', breakpoint_threshold_amount=2)
chunks = text_splitter.split_documents(pages)
print(f'분할된 청크의 수: {len(texts)}')

분할된 청크의 수: 219
